In [ ]:
%load_ext autoreload
%autoreload 2

%store -r the_page
%store -r the_editor
%store -r editions
%store -r calculator
%store -r editor_inputname

editor_editions = editions[editions['editor_id']==the_editor.userid]

# 1. History of editor on a page

In [ ]:
editor_editions = editions[editions['editor_id']==the_editor.userid]

# Visualization
from visualization.editions_listener import DFListener
from ipywidgets import interact
listener = DFListener(editor_editions)
actions = (editor_editions.loc[:,'actions':'actions_stopword_count'].columns.append(
    editor_editions.loc[:,'adds':'reins_stopword_count'].columns)).values.tolist()


interact(listener.editions_per_month, 
         begin=editor_editions.year_month,
         end=editor_editions.year_month.sort_values(ascending=False),
         granularity=['Monthly', 'Yearly'],
         black=actions, 
         red=['actions_surv_48h'] + actions + ['None'],
         green=['None'] + actions, 
         blue=['None'] + actions)

# 2. Tokens that enter into conflict with other editors

In [ ]:
# Visualization
from visualization.wordcloud_listener import WCListener
from ipywidgets import interact
import pandas as pd
    
editor_conflicts = calculator.df[calculator.df['editor']==editor_inputname]
listener = WCListener(editor_conflicts)

df = editor_conflicts
month_year = pd.Series(df['rev_time'].dt.to_period('M').unique()).dt.to_timestamp()

interact(listener.tokens_wordcloud, 
         begin=month_year.sort_values(),
         end=month_year.sort_values(ascending=False),
         action=['Both', 'Just insertions', 'Just Deletions'],
         only_conflict = True)

In [ ]:
from visualization.wordclouder import WordClouder
# from visualization.wordcloud_listener import WCListener
# from ipywidgets import interact
# import pandas as pd

df = calculator.df
df_in = pd.Series(df[df['action']=='in']['token'] + '+')
df_out = pd.Series(df[df['action']=='out']['token'] + '-')
in_out = pd.concat([df_in, df_out])
df["in_out"] = in_out

words = pd.Series(calculator.df.groupby('in_out').size())
colors = {'+': '#00ff00', '-': '#ff0000'}

wc = WordClouder(words, colors)

words

# listener = WCListener(wc)

# month_year = pd.Series(df['rev_time'].dt.to_period('M').unique()).dt.to_timestamp()

# interact(listener.tokens_wordcloud, 
#          begin=month_year.sort_values(),
#          end=month_year.sort_values(ascending=False),
#          action=['Both', 'Just insertions', 'Just Deletions'],
#          only_conflict = True)


In [ ]:
from IPython.display import HTML
HTML('<a href="4. Using WikiWho to analyze Information about conflicting editors.ipynb" target="_blank">Go to next workbook</a>')